In [27]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [28]:
import numpy as np 
import  matplotlib.pyplot as plt 
%matplotlib notebook 
# last line enables interactive plotting in notebooks 

In [1]:
#!python

# -------- these are not just comments. These are compiler directives. 
# They make code run a bit faster. You should take them away for debug/development
# (if you use wraparound=False you can't use negative number for indexing arrays from the end)
#
#cython: boundscheck=False
#cython: wraparound=False
#cython: initializedcheck=True

import numpy as np
import cython    


cohesinAmount = 10
rnapAmount = 5
baseNumber = 500
geneNumber = 10
maxLength = 15
sisterOne = np.empty(cohesinAmount + rnapAmount, dtype=int)
sisterTwo = np.empty(cohesinAmount + rnapAmount, dtype=int)

#This function checks if a location on a chromatid is occupied
#Takes in a sister chromatid and a location on said chromatid
def isOccupied(array, place):
    check = 0
    while check < array.size:
        if place == array[check]:
            return bool(True)
        else:
            check = check + 1
    if check == array.size:
        return bool(False)

#This checks if a stretch of a chromatid is occupied
#Takes in chromatid, location, and the length of stretch after said location that needs to be checked
def extendedOccupied(array, place, length):
    check = 0
    while check < length:
        if isOccupied(array, place+check):
            return bool(True)
        else:
            check = check + 1
    if check == array.size:
        return bool(False)
    
#Generates an array of gene lengths
def generateGeneLengths():
    gene = 0
    geneLengths = np.empty(geneNumber, dtype=int);
    while gene < geneNumber:
        geneLength = np.random.randint(5, maxLength)   
        geneLengths[gene] = geneLength
        gene = gene + 1
    return geneLengths

#Generates an array of the starting points of genes
#Takes in the array of gene lengths to make sure the genes don't overlap
def generateGenes(array):
    gene = 0
    geneList = np.empty(geneNumber, dtype=int);
    while gene < geneNumber:
        geneStart = np.random.randint(0, baseNumber)
        if not extendedOccupied(geneList, geneStart, array[gene]):    
            geneList[gene] = geneStart
            gene = gene + 1
    geneList = np.sort(geneList)
    return geneList

#Generates the starting points of every cohesin on a set chromatid
#Takes in a chromatid
def generateCohesins(array):
    cohesin = 0
    while cohesin < cohesinAmount:
        place = np.random.randint(0, baseNumber)
        if not isOccupied(array, place):
            array[rnapAmount+cohesin] = place
            cohesin = cohesin + 1
    return array

#Generates the starting points of every RNAP on a set chromatid
#Takes in a chromatid
def generateRnaps(array, geneList):
    rnap = 0
    while rnap < rnapAmount:
        gene = np.random.randint(0, 10)
        if not isOccupied(array, geneList[gene]):
            array[rnap] = geneList[gene]
            rnap = rnap + 1
    return array
    
    




In [30]:
geneLengths1 = generateGeneLengths()
geneList1 = generateGenes(geneLengths1)
geneLengths2 = generateGeneLengths()
geneList2 = generateGenes(geneLengths2)
sisterOne = generateCohesins(sisterOne)
sisterTwo = np.copy(sisterOne)
sisterOne = generateRnaps(sisterOne, geneList1)
sisterTwo = generateRnaps(sisterTwo, geneList2)

print(geneLengths1)
print(geneList1)
print(sisterOne)
print(geneLengths2)
print(geneList2)
print(sisterTwo)




[ 5  9  5 13 10 12  8  9  6 13]
[114 151 249 261 277 288 320 348 376 487]
[114 348 487 151 277  32 426 493 250 320  72 232 442 150 344]
[13  9 12  6 14 10  6 14 12 10]
[ 22  75 108 124 155 175 204 271 321 349]
[321 155  75 271 108  32 426 493 250 320  72 232 442 150 344]


In [31]:
speedRnap = .75
speedCohesin = .4

#Every RNAP takes has a certain chance of taking a step forward
#Takes in a chromatid
def stepRnapPositive(array):
    movingRnap = 0
    speed = speedRnap
    while movingRnap < rnapAmount:
        movementCheck = np.random.random()
        if movementCheck < speed:
            array[movingRnap] = array[movingRnap] + 1
            movingRnap = movingRnap + 1
        else:
            movingRnap = movingRnap + 1
    return array

#Every RNAP takes has a certain chance of taking a step backwards
#Takes in a chromatid
def stepRnapNegative(array):
    movingRnap = 0
    speed = speedRanp
    while movingRnap < rnapAmount:
        movementCheck = np.random.random()
        if movementCheck < speed:
            array[movingRnap] = array[movingRnap] - 1
            movingRnap = movingRnap + 1
        else:
            movingRnap = movingRnap + 1
    return array

#Every cohesin has an equal chance of going forward or backwards
#Takes in chromatid
def stepCohesin(array):
    movingCohesin = 0
    speed = speedCohesin
    while movingCohesin < cohesinAmount:
        movementCheck = np.random.random()
        if movementCheck > speed:
            movingCohesin = movingCohesin + 1
        elif movementCheck > speed/2:
            if array[rnapAmount + movingCohesin] < baseNumber:
                array[rnapAmount + movingCohesin] = array[rnapAmount + movingCohesin] + 1
            movingCohesin = movingCohesin + 1
        else:
            if array[rnapAmount + movingCohesin] > 0:
                array[rnapAmount + movingCohesin] = array[rnapAmount + movingCohesin] - 1
            movingCohesin = movingCohesin + 1
    return array


In [32]:
print(sisterOne)
sisterOne = stepRnapPositive(sisterOne)
sisterOne = stepCohesin(sisterOne)
print(sisterOne)

[114 348 487 151 277  32 426 493 250 320  72 232 442 150 344]
[114 348 488 152 277  32 426 492 250 320  73 232 443 150 344]
